In [1]:
# !pip install transformers accelerate
# !pip install transformers[sentencepiece]
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
# !pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/miniconda3/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
label2id = {'Adaptive':0,'Perfective':1,'Corrective':2}

In [3]:
train_II = pd.read_csv(r'dataset II/train.csv', encoding='utf_8_sig')
train_II.fillna('', inplace=True)
train_II = train_II[['label','text']].reset_index(drop=True)
test_II = pd.read_csv(r'dataset II/test.csv', encoding='utf_8_sig')
test_II.fillna('', inplace=True)
test_II = test_II[['label','text']].reset_index(drop=True)

In [4]:
train_II

,label,text
0,Adaptive,Trying to extend the Scheduler interface accor...
1,Corrective,[DROOLS-839] fix LogicTransformer with Accumul...
2,Adaptive,"libvaladoc/html: Add ""All known members inheri..."
3,Adaptive,[AESH-316] - print warning instead of exceptio...
4,Perfective,remove unused private field\n
...,...,...
1250,Adaptive,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...
1251,Adaptive,"search is more complete, support forward/rever..."
1252,Corrective,"Extract Function: Consider reference ""broken"" ..."
1253,Perfective,First part of https://jira.duraspace.org/brows...


In [5]:
train = train_II
train = train.replace({'label':label2id})
test = test_II
test = test.replace({'label':label2id})

In [6]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test.iterrows():
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)

# for index, row in test.iterrows():
#     input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
#     dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "Trying to extend the Scheduler interface according- to the comments at -19.--",
  "text_b": "",
  "tgt_text": null
}



In [7]:
len(train)

1255

In [8]:
len(test)

538

In [9]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Trying to extend the Scheduler interface according- to the comments at -19.--', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [11]:
# WrapperClass.save_pretrained(nb_fname)

In [12]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 3, 26421, 12, 4285, 8, 14890, 52, 3459, 1315, 18, 12, 8, 2622, 44, 3, 4481, 5, 18, 18, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁', 'Trying', '▁to', '▁extend', '▁the', '▁Schedule', 'r', '▁interface', '▁according',

In [13]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (1437 > 512). Running this sequence through the model will result in indexing errors


In [14]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 1255it [00:02, 492.88it/s]


In [15]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=3).from_file("knowledgeable_verbalizer-II.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [19, 20, 20]
Parameter containing:
tensor([[[ 3374,  1528,     0,     0,     0],
         [14340,   138,     0,     0,     0],
         [    3, 21007,    51,  7360,  1528],
         [ 6142,   757,     0,     0,     0],
         [ 1460, 13376,   257,     0,     0],
         [ 4639,     0,     0,     0,     0],
         [ 3374,   179,     0,     0,     0],
         [ 9009,     0,     0,     0,     0],
         [  433,     0,     0,     0,     0],
         [27679,    53,     0,     0,     0],
         [10152,     0,     0,     0,     0],
         [11887,     0,     0,     0,     0],
         [ 7916,     0,     0,     0,     0],
         [18149,     0,     0,     0,     0],
         [  529,   747,   291,     0,     0],
         [27875,     0,     0,     0,     0],
         [15208,     0,     0,     0,     0],
         [ 6624,     0,     0,     0,     0],
         [12368,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0]],

      

In [16]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [17]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
import ipynbname
nb_fname = ipynbname.name()
print(nb_fname)

PKG_II


In [19]:
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 538it [00:01, 502.32it/s]


In [20]:
# Evaluate
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0

for epoch in tqdm(range(10)):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        torch.save(prompt_model.state_dict(), nb_fname +'.pth')
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, average loss: 3.047637701034546
Epoch 0, average loss: 0.9616746718392652
Epoch 0, average loss: 0.7831344527404497
Epoch 0, average loss: 0.6867654867692302
Epoch 0, validation loss: 0.4982205253508356


 10%|█         | 1/10 [01:14<11:09, 74.39s/it]

Epoch 1, average loss: 0.3063117563724518
Epoch 1, average loss: 0.3299380229676471
Epoch 1, average loss: 0.33299975754637823
Epoch 1, average loss: 0.30928517247099946
Epoch 1, validation loss: 0.5030378891931225


 20%|██        | 2/10 [02:30<10:03, 75.45s/it]

Epoch 2, average loss: 0.039146097376942635
Epoch 2, average loss: 0.1751489688952764
Epoch 2, average loss: 0.1808203580687837
Epoch 2, average loss: 0.17626853968866404
Epoch 2, validation loss: 0.6300758784606674


 30%|███       | 3/10 [03:46<08:51, 75.87s/it]

Epoch 3, average loss: 0.394154311157763
Epoch 3, average loss: 0.11534453734445076
Epoch 3, average loss: 0.10390706908702389
Epoch 3, average loss: 0.10580542141826829
Epoch 3, validation loss: 0.657430423626728
Early stopping at epoch 3.


 30%|███       | 3/10 [05:05<11:52, 101.80s/it]


In [21]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
# 这里是数字所以要转字符
class_names = list(test['label'].unique())  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, digits=4)

print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.8289    0.8757    0.8516       177
           1     0.8430    0.8056    0.8239       180
           2     0.8324    0.8232    0.8278       181

    accuracy                         0.8346       538
   macro avg     0.8348    0.8348    0.8344       538
weighted avg     0.8348    0.8346    0.8343       538



In [22]:
# store the checkpoints
prompt_model.plm.save_pretrained(nb_fname)
model_config.save_pretrained(nb_fname)
myverbalizer.tokenizer.save_pretrained(nb_fname)

('PKG_II/tokenizer_config.json',
 'PKG_II/special_tokens_map.json',
 'PKG_II/spiece.model',
 'PKG_II/added_tokens.json')

In [23]:
# import os
# os.system('shutdown')